In [ ]:
from nba_api.stats.endpoints import playbyplayv2

# 比赛ID示例
game_id = '0011600051'  # 替换为目标比赛ID

# 获取比赛逐回合数据
play_by_play = playbyplayv2.PlayByPlayV2(game_id=game_id)
data = play_by_play.get_data_frames()[0]  # 返回DataFrame格式数据
last_quater = data.iloc[-1]["PERIOD"]
# 筛选第四节数据
last_quater_data = data[data['PERIOD'] == last_quater]
last_quater_data = last_quater_data[last_quater_data["SCOREMARGIN"].notna()]
last_quater_data.to_csv("last_quater_data.csv", index=False)



In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable="2019-20")  # 2019-20赛季包含2020年数据
gamefinder = leaguegamefinder.LeagueGameFinder(game_id_nullable="0022400340")  # 2019-20赛季包含2020年数据
games = gamefinder.get_data_frames()[0]
games.to_csv("games.csv", index=False)
print(len(games))
# target_games = games[games["GAME_ID"]=="0041900213"]

# # # 筛选2020年的比赛
# # games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
# # games_2020 = games[games['GAME_DATE'].dt.year == 2020]
# # print(games_2020[["TEAM_NAME", "GAME_ID", "PTS", "MIN"]].head())

# # # 提取赢家信息
# winners = target_games[['GAME_DATE', 'MATCHUP', 'WL']]
# # # print(winners.head())
# winners['WINNER'] = winners.apply(
#     lambda row: row['MATCHUP'].split(" ")[0] if row['WL'] == 'W' else row['MATCHUP'].split(" ")[-1],
#     axis=1
# )
# print(winners)

# # 输出前几场比赛的赢家
# # print(winners[['GAME_DATE', 'MATCHUP', 'WINNER']].head())
# # 输出比分情况
# print(winners['MATCHUP'].head())



1


In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# 获取2019-20赛季的所有比赛数据
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable="2019-20")  # 2019-20赛季
games = gamefinder.get_data_frames()[0]

# 筛选2020年的比赛
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games_2020 = games[games['GAME_DATE'].dt.year == 2020]
print(games_2020.keys())
# 提取比赛日期、对阵、比分和胜负信息
games_2020 = games_2020[['GAME_DATE', 'MATCHUP', 'PTS', 'PTS_OPP', 'WL']]

# # 重命名列以便于理解
# games_2020.rename(columns={
#     'PTS': 'TEAM_SCORE',
#     'PTS_OPP': 'OPPONENT_SCORE'
# }, inplace=True)

# 添加赢家列
games_2020['WINNER'] = games_2020.apply(
    lambda row: row['MATCHUP'].split(" ")[0] if row['WL'] == 'W' else row['MATCHUP'].split(" ")[-1],
    axis=1
)

# 显示前几场比赛的最终比分和赢家
print(games_2020[['GAME_DATE', 'MATCHUP', 'PTS', 'PTS_OPP', 'WINNER']].head())


In [ ]:
from nba_api.stats.endpoints import leaguegamefinder, playbyplayv2
import pandas as pd
from tqdm import tqdm

# 获取2020赛季的比赛数据
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable="2020-21")
games = gamefinder.get_data_frames()[0]

# 提取需要的字段
games = games[['GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[:1]
# 初始化存储结果
results = []

# 遍历每场比赛
for _, game in tqdm(games.iterrows(), total=games.shape[0], desc="Processing games"):
    game_id = game['GAME_ID']
    matchup = game['MATCHUP']
    winner = game['MATCHUP'].split(' ')[0] if game['WL'] == 'W' else game['MATCHUP'].split(' ')[-1]
    
    # 获取逐回合数据
    play_by_play = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]

    # 筛选得分事件
    scoring_events = play_by_play[play_by_play['EVENTMSGTYPE'] == 1]

    # # 解析 SCORE 字段为主客队得分
    # scoring_events['HOME_SCORE'], scoring_events['AWAY_SCORE'] = zip(
    #     *scoring_events['SCORE'].fillna("0-0").apply(lambda x: map(int, x.split('-')))
    # )

    # 转换 PCTIMESTRING 为秒数
    scoring_events['PCTIMESTRING'] = scoring_events['PCTIMESTRING'].str.split(":").apply(
        lambda x: int(x[0]) * 60 + int(x[1]) if isinstance(x, list) and len(x) == 2 else 0
    )
    play_by_play['PERIOD'] = play_by_play['PERIOD'].astype(int)

    # 筛选最后五分钟的数据
    last_five_minutes = play_by_play[(play_by_play['PCTIMESTRING'] <= 300) & (play_by_play['PERIOD'] == 4)]
    socre_diff = int(last_five_minutes["SCORE"].split("-")[0]) - int(last_five_minutes["SCORE"].split("-")[1])
    last_five_minutes["SCORE_DIFF"] = socre_diff
    # save last_five_minutes
    last_five_minutes.to_csv("last_five_minutes.csv", index=False)
    # # 计算最后五分钟内的得分（差值）
    # if not last_five_minutes.empty:
    #     home_score_last5 = last_five_minutes['HOME_SCORE'].iloc[-1] - last_five_minutes['HOME_SCORE'].iloc[0]
    #     away_score_last5 = last_five_minutes['AWAY_SCORE'].iloc[-1] - last_five_minutes['AWAY_SCORE'].iloc[0]
    # else:
    #     home_score_last5, away_score_last5 = 0, 0

    # results.append({
    #     "GAME_DATE": game['GAME_DATE'],
    #     "MATCHUP": matchup,
    #     "WINNER": winner,
    #     "HOME_SCORE_LAST5": home_score_last5,
    #     "AWAY_SCORE_LAST5": away_score_last5
    # })

# 转换为DataFrame
results_df = pd.DataFrame(results)

# 显示结果
print(results_df.head())

# 可选：保存为CSV
results_df.to_csv("nba_last5min_scores.csv", index=False)
